# NFL Passrush/Blocker Section 1 (of n)
# Creating the base play frame

This workbook is where the functions to build a play frame up to metrics creation, which is found in the next workbook.

#### Libraries Used

In [1]:
import nfl_acquire_and_prep as acquire
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')

#### Generate Random play for testing

In [2]:
def random_play():
    '''
    Creates a random play to analyze from the entire 8-week season.
    Play numbers can be assigned in different weeks, so they are not unique, hence the need to specify game.

    Parameters:
        NONE
    Returns:
        'game' - Integer - Game number (unique for season)
        'play' - Integer - Unique for game
    '''
    # Pull in data from acquire function that creates an object full of game-play combinations
    game_play_players = acquire.all_plays()

    # Randomly choose a game then a play from that game
    game = np.random.choice(list(game_play_players.keys()))
    play = np.random.choice(list(game_play_players[game]))

    return game, play

In [3]:
game, play = random_play()
game, play

(2021102410, 3714)

#### Or choose a specific play

----

## 1. Create frame of play with ball and all players

#### Support Functions: 

i. Get week number from game:

In [4]:
def get_week_of_game(game):
    '''
    Gets the week number of a given game, which is needed to access the proper weekly data csv.

    Parameters:
        'game' - Integer - The unique number given to the game

    Returns: 
        'week_num' - Integer - The week number for a given game
    '''
    # Acquire small df with game + week information
    games = acquire.games()

    # Match the game to a week
    week_num = games.loc[games.game == game, 'week']

    return int(week_num)

In [5]:
week_num = get_week_of_game(game)
week_num

7

ii. Add previous location data

In [6]:
def add_next(play_frames_df):
    '''
    Adds in the next location (x and y coordinates) to use for metric building.

    Parameters:
        'play_frames_df' - Dataframe - Play player movement data for a given play

    Returns:
        'play_frames_df' - Dataframe - Play player movement data for a given play
    '''
    # Create new column with x and y locations shifted one frame forward
    play_frames_df['next_x'] = play_frames_df.x.shift(-1)
    play_frames_df['next_y'] = play_frames_df.y.shift(-1)
    
    return play_frames_df

#### Core Function:

In [7]:
def get_play_frames(game, play):
    '''
    Takes in a game and play and returns the movement of all players plus the ball in .1 second frames 
    over the course of the play.  Necessary to create metrics that rely on quasi-continuous data.

    Parameters:
        'game' - Integer - Game number (unique for season)
        'play' - Integer - Unique for game
    Returns: 
        'play_frames_df' - Dataframe - All players (nflId) and the ball (nflId = 0) and their movement data.
    '''
    # Find the proper week to analyze and bring that dataframe in
    week_num = get_week_of_game(game)

    # Acquire that week's df
    week_df = acquire.week(week_num)

    # Extract frames for a given play
    play_frames_df = week_df[week_df.game == game][week_df.play == play]
    
    # Add in the shifts
    play_frames_df = add_next(play_frames_df)

    return play_frames_df

#### Core function test:

In [8]:
play_frames_df = get_play_frames(game, play)
play_frames_df

,game,play,nflId,frame,x,y,s,a,dis,o,dir,event,next_x,next_y
828460,2021102410,3714,38547,1,30.38,41.38,0.16,0.21,0.02,156.94,123.17,None,30.39,41.37
828461,2021102410,3714,38547,2,30.39,41.37,0.18,0.12,0.02,156.94,121.00,None,30.42,41.36
828462,2021102410,3714,38547,3,30.42,41.36,0.23,0.07,0.03,158.14,117.35,None,30.44,41.34
828463,2021102410,3714,38547,4,30.44,41.34,0.26,0.04,0.03,161.15,124.30,None,30.48,41.32
828464,2021102410,3714,38547,5,30.48,41.32,0.31,0.06,0.04,162.78,122.00,None,30.51,41.31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
829444,2021102410,3714,0,39,46.15,24.69,1.28,1.95,0.21,0.00,0.00,None,42.91,27.01
829445,2021102410,3714,0,40,42.91,27.01,19.53,1.14,3.99,0.00,0.00,None,41.35,28.19
829446,2021102410,3714,0,41,41.35,28.19,19.38,1.97,1.96,0.00,0.00,None,39.81,29.36
829447,2021102410,3714,0,42,39.81,29.36,19.20,2.47,1.94,0.00,0.00,None,38.29,30.53


----

## 2. Create frames with ball and player(s) of interest

#### Support Functions:

i. Clean football frames

In [9]:
def clean_fb_frames(play_fb_frames):
    '''
    Cleans up the football frames (nflId = 0) of the play movement dataframe.

    Parameters:
        'play_fb_frames' - Dataframe - Football movement over play, 'dirty'
    Returns:
        'play_fb_frames' - Dataframe - Football movement over play, now cleaned
    '''
    # Drop unecessary columns, and rename remaining
    play_fb_frames = play_fb_frames.drop(columns = ['game',
                                                    'play',
                                                    'nflId',
                                                    's',
                                                    'o',
                                                    'a',
                                                    'dis',
                                                    'dir']).rename(columns = {'x':'ball_x',
                                                                              'y':'ball_y',
                                                                              'next_x':'ball_next_x',
                                                                              'next_y':'ball_next_y'})
    # Re-order columns
    play_fb_frames = play_fb_frames[['event',
                                     'ball_x',
                                     'ball_y',
                                     'ball_next_x',
                                     'ball_next_y']]
    
    return play_fb_frames

ii. Clean player frames

In [10]:
def clean_player_frames(play_player_frames):
    '''
    Cleans up the player frames (nflId = 0) of the play movement dataframe.

    Parameters:
        'play_player_frames' - Dataframe - Player movement over play, 'dirty'
    Returns:
        'play_player_frames' - Dataframe - Player movement over play, now cleaned
    '''
    # Drop unecessary columns, and rename remaining
    play_player_frames = play_player_frames.drop(columns = ['game',
                                                            'play',
                                                            's',
                                                            'o',
                                                            'dir',
                                                            'dis',
                                                            'event']).rename(columns = {'x':'player_x',
                                                                                        'y':'player_y',                                                                    
                                                                                        'a':'player_a',
                                                                                        'next_x':'player_next_x',
                                                                                        'next_y':'player_next_y'})
    # Re-order columns
    play_player_frames = play_player_frames[['nflId',
                                             'player_x',
                                             'player_y',
                                             'player_next_x',
                                             'player_next_y',
                                             'player_a']]
    
    return play_player_frames

iii. >Optional< Determine Pertinent Frames (to truncate off of)

In [11]:
def determine_pertinent_frames(play_fb_frames):
    '''
    This function determines the indices of the relevant frames for pass rush analysis.
    It parses the frame events for a starting (snap) index and the ending index for the analysis.
    
    Parameters:
        'play_fb_frames' - Dataframe - Dataframe of fb movement (post-cleaning)
    Returns:
        'snap_index' - Integer - Starting point for truncated frames (the 'snap' event)
        'end_index' - Integer - End point for truncated frames (after ball leave qb event: see list in function)
    '''
    # Initiate a trigger which tells the function to start checking the events after ball snap for the end event (when qb passes, is sacked, etc.)
    trigger = 0
    
    for i, event in enumerate(play_fb_frames.event):
        # If the event is a ball_snap, stores the snap index and triggers the function to start checking events
        if event == 'ball_snap':
            snap_index = i + 1
            trigger = 1
            continue
        
        if trigger == 1:
            # The following events are not end events
            if event in ['None','autoevent_ballsnap','autoevent_passforward','play_action','first_contact','shift','man_in_motion','line_set']:
                continue
            # If the trigger is on and the event is an end event, return the index
            else:
                return snap_index, i + 1 #-> i + 1 being the end index
        else:
            continue

iv. Get football play frames

In [12]:
def get_play_fb_frames(play_frames_df):
    '''
    Isolates the football frames (football movement over the course of the play) for a given play.
    
    Parameters:
        'play_frames_df' - Dataframe - All players (nflId) and the ball (nflId = 0) and their movement data.
    Returns:
        'qb_hold_time' - Float - How long the qb holds the ball (in seconds)
        'point_of_scrimmage' - Tuple of Floats - x and y coordinates of snap
        'play_fb_frames' - Dataframe - Cleaned frames ready to index players off of     
    '''
    # nflId for the football is 0, creates a dataframe of the football over the play
    play_fb_frames = play_frames_df[play_frames_df.nflId == 0].set_index('frame',drop = True)

    # Get the x value for the line of scrimmage (for use with play graphing)
    point_of_scrimmage = (play_fb_frames.x.iloc[0], play_fb_frames.y.iloc[0])
    
    # Clean the frames
    play_fb_frames = clean_fb_frames(play_fb_frames)
    
    # Drop the last frame since the shifted columns will have Nulls
    play_fb_frames = play_fb_frames[:-1]
    
    # Get the start and end frames of the qb with the ball - allows for qb hold time even if truncate = False
    snap_frame, end_frame = determine_pertinent_frames(play_fb_frames)
    
    # Calculate qb hold time
    qb_hold_time = (end_frame - snap_frame)/10
    
    # Remove all frames before snap and after event where ball is no longer being pass rushed
    play_fb_frames = play_fb_frames[(play_fb_frames.index >= snap_frame) & (play_fb_frames.index <= end_frame)]

    # Drops event, as not longer necessary
    play_fb_frames = play_fb_frames.drop(columns = ['event'])

    return qb_hold_time, point_of_scrimmage, play_fb_frames

In [13]:
qb_hold_time, point_of_scrimmage, play_fb_frames = get_play_fb_frames(play_frames_df)
qb_hold_time, point_of_scrimmage, play_fb_frames.head()

(3.1,
 (38.31, 23.87),
        ball_x  ball_y  ball_next_x  ball_next_y
 frame                                          
 7       39.11   23.74        39.66        23.73
 8       39.66   23.73        40.21        23.73
 9       40.21   23.73        40.76        23.71
 10      40.76   23.71        41.29        23.69
 11      41.29   23.69        41.76        23.66)

In [14]:
play_fb_frames.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32 entries, 7 to 38
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ball_x       32 non-null     float64
 1   ball_y       32 non-null     float64
 2   ball_next_x  32 non-null     float64
 3   ball_next_y  32 non-null     float64
dtypes: float64(4)
memory usage: 1.2 KB


v. Get list of players in play (***not used until later):

In [15]:
def get_players_in_play(game, play):
    '''
    Gets all of the NFL player ids along with their role and position for a given play.  
    
    Parameters:
        'game' - Integer - Game number (unique for season)
        'play' - Integer - Unique for game
    Returns:
        'play_players' - Dataframe - A dataframe of play players and their roles and positions
    '''
    # Load data
    scout_players = pd.read_csv('data/pffScoutingData.csv')

    # Clean and rename
    play_players = scout_players[['gameId',
                                  'playId',
                                  'nflId',
                                  'pff_role',
                                  'pff_positionLinedUp']].rename(columns = {'gameId':'game',
                                                                            'playId':'play',
                                                                            'pff_role':'role',
                                                                            'pff_positionLinedUp':'position'})

    # Isolate players from given game and play
    play_players = play_players[play_players.game == game][play_players.play == play].drop(columns = ['game',
                                                                                                      'play']).set_index('nflId')

    return play_players

In [16]:
players = get_players_in_play(game, play)
players

,role,position
nflId,,
38547,Coverage,LCB
38673,Coverage,RCB
39947,Pass Block,LT
40641,Pass Block,LWR
41249,Pass Rush,LOLB
41256,Coverage,SCBiL
42030,Coverage,SCBoL
42360,Pass Rush,LE
42765,Pass Block,RG


vi. Find blocking matchups

In [17]:
def matchup_finder(game, play, nflId, player_type):
    '''
    Used in PvP find the pass blocker(s) blocking the pass rusher, or the pass rusher opposing the blockers
    
    Parameters:
        'game' - Integer - Game number (unique for season)
        'play' - Integer - Unique for game
        'nflId' - Integer - The unique id for the primary player being analyzed
        'player_type' - String - The type of player bring analyzed (pass_rusher or pass_blocker)
    Returns: 
        'player_type' - String - The type of player bring analyzed (pass_rusher or pass_blocker)
        'opponent_type' - String - The type of player opposing the analyzed player
        'opponent_list' - List of ints - List of nflId ints (players) pass rushing/blocking pass rusher
    '''
    scout_pass_block = acquire.scout_pass_block()
    
    scout_pass_block_play = scout_pass_block[scout_pass_block.game == game][scout_pass_block.play == play]
    
    # If player's nflId is in this list, they are a pass blocker and opponent is pass rusher
    if nflId in list(scout_pass_block_play.nflId):
        # Input check of player_type
        if player_type == 'pass_rusher':
            print('Error: Player types do not match.  Check your the player type you inputted as a parameter')
            return

        opponent_type = 'pass_rusher'
        
        return player_type, opponent_type, list(scout_pass_block_play[scout_pass_block_play.nflId == nflId].rusher_blocked)
    
    elif nflId in list(scout_pass_block_play.rusher_blocked):
        if player_type == 'pass_blocker':
            print('Error: Player types do not match.  Check your the player type you inputted as a parameter')
            return 
            
        opponent_type = 'pass_blocker'
        
        return player_type, opponent_type, list(scout_pass_block_play[scout_pass_block_play.rusher_blocked == nflId].nflId)
    
    # In order to not error out in later function if there are no matchups, need to return a blank string and blank list
    else:
        return player_type, '', []

In [18]:
players = get_players_in_play(game, play)
nflId = np.random.choice(list(players[(players.role == 'Pass Block') | (players.role == 'Pass Rush')].index))
player_type, opponent_type, opponent_list =  matchup_finder(game, play, nflId, 'pass_rusher')

In [19]:
player_type, opponent_type, opponent_list

('pass_rusher', 'pass_blocker', [46275])

In [20]:
nflId

41249

vii. Return player frames

In [21]:
def get_play_player_frames(play_frames_df, nflId):
    '''
    Creates a dataframe which isolates the player's frame by frame movement data.
    
    Parameters:
        'play_frames_df' - Dataframe - All players (nflId) and the ball (nflId = 0) and their movement data.
        'nflId' - Integer - Unique Id of player being analyzed
    Returns:
        'player_frames_df' - Dataframe - Frames of selected player
    '''
    # Merge will truncate this if needed later
    player_frames_df = play_frames_df[play_frames_df.nflId == nflId].set_index('frame',drop = True)
    
    # Clean the frames
    player_frames_df = clean_player_frames(player_frames_df)
    
    # Drop the last row, as it is corrupted by the player before it
    player_frames_df = player_frames_df[:-1]

    return player_frames_df

In [22]:
player_frames_df = get_play_player_frames(play_frames_df, nflId)
player_frames_df

,nflId,player_x,player_y,player_next_x,player_next_y,player_a
frame,,,,,,
1,41249,37.14,29.99,37.14,29.99,0.09
2,41249,37.14,29.99,37.15,30.01,0.08
3,41249,37.15,30.01,37.15,30.01,0.08
4,41249,37.15,30.01,37.16,30.00,0.07
5,41249,37.16,30.00,37.17,30.01,0.06
6,41249,37.17,30.01,37.18,30.04,0.06
7,41249,37.18,30.04,37.22,30.03,0.11
8,41249,37.22,30.03,37.27,30.03,0.24
9,41249,37.27,30.03,37.34,30.01,0.80


viii. merge frames

In [23]:
def merge_frames(df1, df2):
    '''
    Puts together different frames generated from the weekly data.
    Note: Ensure you always use the football frames as the leftmost df1 if you have multiple players to merge.
    
    Parameters:
        'df1' - Dataframe - Left frame to merge on
        'df2' - Dataframe - Right frame to merge on df1
    Returns:
        'merged_frames' - Dataframe - df1 and df2 merged on 'frame' column
    '''
    merged_frames = df1.merge(df2, on = 'frame', how = 'left')
    
    return merged_frames

Rename player analyzed


In [24]:
def rename_player(player_frames_df, player_type):
    '''
    Renames the main player being analyzed by typing them (pass rusher or blocker)
    
    Parameters:
        'player_frames_df' - Dataframe - Frames of selected player
        'player_type' - String - The type of player bring analyzed (pass_rusher or pass_blocker)
    Returns:
        'player_frames_df' - Dataframe - Frames of selected player (now renamed to player type)

    '''
    player_frames_df = player_frames_df.rename(columns = {'nflId':f'{player_type}',
                                                          'player_a':f'{player_type}_a',
                                                          'player_x':f'{player_type}_x',
                                                          'player_y':f'{player_type}_y',
                                                          'player_next_x':f'{player_type}_next_x',
                                                          'player_next_y':f'{player_type}_next_y'})
    
    return player_frames_df

ix. Rename opponent frames

In [25]:
def rename_opponents(opponent_frames, opponent_count, opponent_type, count = 0):
    '''
    Renames opposing players to make understanding and manipulations easier.
    
    Parameters:
        'opponent_frames' - Dataframe - Play frames of the opponent
        'opponent_count' - Integer - How many opponents will be analyzed
        'opponent_type' - String - Like player_type, is 'pass_rusher' or 'pass_blocker'
    Returns:
        'opponent_frames' - Dataframe - Play frames of the opponent (now with new names)
    '''
    # If there is a single opponent
    if opponent_count == 1:
        #Change the column names to nflId and opponent to make easier
        opponent_frames = opponent_frames.rename(columns = {'nflId':f'{opponent_type}',
                                                            'player_x':f'{opponent_type}_x',
                                                            'player_y':f'{opponent_type}_y',
                                                            'player_a':f'{opponent_type}_a',
                                                            'player_next_x':f'{opponent_type}_next_x',
                                                            'player_next_y':f'{opponent_type}_next_y'})

    else:
        opponent_frames = opponent_frames.rename(columns = {'nflId':f'{opponent_type}_{count}',
                                                            'player_x':f'{opponent_type}_{count}_x',
                                                            'player_y':f'{opponent_type}_{count}_y',
                                                            'player_a':f'{opponent_type}_{count}_a',
                                                            'player_next_x':f'{opponent_type}_{count}_next_x',
                                                            'player_next_y':f'{opponent_type}_{count}_next_y'})
        
    return opponent_frames

#### Core Function: 

In [26]:
def create_play_analysis_frames(play_frames_df, nflId, player_type, v_type): # else?
    '''
    Create the play frames, consisting of the curent and next x and y coordiantes, as well as the acceleration
    at the time of the frame, for the ball and pass rusher, as well as opponents if PvP is selected.
    PvB is used only to analyze pass rushers without accounting for any blockers.
    
    Parameters:
        'play_frames_df' - Dataframe - All players (nflId) and the ball (nflId = 0) and their movement data.
        'nflId' - Integer - Unique Id of player being analyzed
        'player_type' - String - The type of player bring analyzed (pass_rusher or pass_blocker)
        'v_type' - String - The type of analysis to perform, player vs ball or player vs player (and ball.)  Either 'PvB' or 'PvP'.
    Returns:
        'qb_hold_time' - Float - How long the qb holds the ball (in seconds)
        'point_of_scrimmage' - Tuple of Floats - x and y coordinates of snap
        'play_fb_frames' - Dataframe - Cleaned frames ready to index players off of 
    '''
    # Get football frames: used for all
    qb_hold_time, point_of_scrimmage, play_fb_frames = get_play_fb_frames(play_frames_df)
    
    # Get game and play
    game = int(play_frames_df.game.unique())
    play = int(play_frames_df.play.unique())

    # Use the matchup function to see what kind of player the NFL ID is, and who they went against in the scouting data
    player_type, opponent_type, opponents = matchup_finder(game, play, nflId, player_type)
    
    # Get the number of opponents (used for labeling)
    opponent_count = len(opponents)
    
    if v_type == 'PvB':
        # Get player
        player_frames_df = get_play_player_frames(play_frames_df, nflId)
        
        # Rename player analyze to pass_rusher or pass_blocker
        player_frames_df = rename_player(player_frames_df, player_type)
        
        # Create a dataframe which pairs the location and movement with the football and the player frame by frame (0.1 second intervals)
        pvb_analysis_frames = merge_frames(play_fb_frames, player_frames_df)
        
        # Drop first and last frames - while losing a small amount of data improves overall analysis
        pvb_analysis_frames = pvb_analysis_frames[1:-1]
        
        # Reset frames
        pvb_analysis_frames = pvb_analysis_frames.reset_index(drop = True)

        return qb_hold_time, point_of_scrimmage, pvb_analysis_frames        
    
    elif v_type == 'PvP':
        # Get player
        player1_frames_df = get_play_player_frames(play_frames_df, nflId)
        
        # Rename player analyzed
        player1_frames_df = rename_player(player1_frames_df, player_type)
        
        # Create a dataframe which pairs the location and movement with the football and the player frame by frame (0.1 second intervals)
        pvp_analysis_frames = merge_frames(play_fb_frames, player1_frames_df)
    
        if opponent_count == 1:
            
            player = opponents[0]
        
            opponent_frames = get_play_player_frames(play_frames_df, player)
            
            pvp_analysis_frames = merge_frames(pvp_analysis_frames, opponent_frames)
            
            # Rename opponent
            pvp_analysis_frames = rename_opponents(pvp_analysis_frames, opponent_count, opponent_type)
    
        elif opponent_count > 1:
            
            # Create a counter for the rename function to keep track of
            count = 0
            
            # Now use this list to add an integer lable to each opponent's columns
            for player in opponents:
                # Up the counter for the rename function
                count += 1
                opponent_frames = get_play_player_frames(play_frames_df, player)
                
                opponent_frames = rename_opponents(opponent_frames, opponent_count, opponent_type, count = count)

                pvp_analysis_frames = merge_frames(pvp_analysis_frames,opponent_frames)     
        
        else:
            print('Player did not have any opponents listed, cannot do PvP, showing PvB instead')            
        
        # Drop first and last frames - while losing a small amount of data improves overall analysis
        pvp_analysis_frames = pvp_analysis_frames[1:-1] 
        
        # Reset frames
        pvp_analysis_frames = pvp_analysis_frames.reset_index(drop = True)
        
        return qb_hold_time, point_of_scrimmage, pvp_analysis_frames
        
    else:
        print('v_type inputs are either:\n-"PvB" to compare pass rusher to ball/QB; or:\n-"PvP" to compare pass rusher to blocker')

#### Core Function Test:

In [27]:
qb_hold_time, point_of_scrimmage, pvp_analysis_frames = create_play_analysis_frames(play_frames_df, nflId, player_type, v_type = 'PvP')


In [28]:
qb_hold_time

3.1

In [29]:
point_of_scrimmage

(38.31, 23.87)

In [30]:
pvp_analysis_frames

,ball_x,ball_y,ball_next_x,ball_next_y,pass_rusher,pass_rusher_x,pass_rusher_y,pass_rusher_next_x,pass_rusher_next_y,pass_rusher_a,pass_blocker,pass_blocker_x,pass_blocker_y,pass_blocker_next_x,pass_blocker_next_y,pass_blocker_a
0,39.66,23.73,40.21,23.73,41249,37.22,30.03,37.27,30.03,0.24,46275,39.47,27.00,39.49,27.02,0.00
1,40.21,23.73,40.76,23.71,41249,37.27,30.03,37.34,30.01,0.80,46275,39.49,27.02,39.50,27.02,0.00
2,40.76,23.71,41.29,23.69,41249,37.34,30.01,37.44,30.01,1.41,46275,39.50,27.02,39.50,27.02,0.02
3,41.29,23.69,41.76,23.66,41249,37.44,30.01,37.63,30.00,1.94,46275,39.50,27.02,39.56,27.05,0.35
4,41.76,23.66,42.18,23.64,41249,37.63,30.00,37.84,29.98,2.47,46275,39.56,27.05,39.70,27.10,2.16
5,42.18,23.64,42.53,23.63,41249,37.84,29.98,38.11,29.97,2.30,46275,39.70,27.10,39.85,27.16,4.55
6,42.53,23.63,42.88,23.62,41249,38.11,29.97,38.41,29.94,2.90,46275,39.85,27.16,40.09,27.26,4.02
7,42.88,23.62,43.19,23.60,41249,38.41,29.94,38.80,29.86,2.96,46275,40.09,27.26,40.35,27.38,4.50
8,43.19,23.60,43.46,23.58,41249,38.80,29.86,39.19,29.79,2.66,46275,40.35,27.38,40.64,27.49,3.67
9,43.46,23.58,43.68,23.57,41249,39.19,29.79,39.62,29.73,2.14,46275,40.64,27.49,40.93,27.63,3.79


----

## 3. Combine the two Core Functions

In [31]:
def build_play_frames(game, play, nflId, player_type, v_type = 'PvB'):
    '''
    Combines the function which creates the play frames with the function that creates the play frames.
    
    Parameters:
        'game' - Integer - Game number (unique for season)
        'play' - Integer - Unique for game
        'nflId' - Integer - Unique Id of player being analyzed
        'player_type' - String - The type of player bring analyzed (pass_rusher or pass_blocker)
        'v_type' - String - The type of analysis to perform, player vs ball or player vs player (and ball.)  Either 'PvB' or 'PvP'.
    Returns:
        'qb_hold_time' - Float - How long the qb holds the ball (in seconds)
        'point_of_scrimmage' - Tuple of Floats - x and y coordinates of snap
        'play_fb_frames' - Dataframe - Cleaned frames ready to index players off of 
    '''
    play_frames_df = get_play_frames(game, play)
    
    qb_hold_time, point_of_scrimmage, analysis_frames = create_play_analysis_frames(play_frames_df,
                                                                                    nflId, 
                                                                                    player_type = player_type,
                                                                                    v_type = v_type)
    
    return qb_hold_time, point_of_scrimmage, analysis_frames

In [32]:
qb_hold_time, point_of_scrimmage, analysis_frames = build_play_frames(game, play, nflId, player_type, v_type = 'PvB')

In [33]:
qb_hold_time

3.1

In [34]:
print(f'line of scrimmage = {point_of_scrimmage[0]}\ny-coord of ball = {point_of_scrimmage[1]}')

line of scrimmage = 38.31
y-coord of ball = 23.87


In [35]:
analysis_frames

,ball_x,ball_y,ball_next_x,ball_next_y,pass_rusher,pass_rusher_x,pass_rusher_y,pass_rusher_next_x,pass_rusher_next_y,pass_rusher_a
0,39.66,23.73,40.21,23.73,41249,37.22,30.03,37.27,30.03,0.24
1,40.21,23.73,40.76,23.71,41249,37.27,30.03,37.34,30.01,0.80
2,40.76,23.71,41.29,23.69,41249,37.34,30.01,37.44,30.01,1.41
3,41.29,23.69,41.76,23.66,41249,37.44,30.01,37.63,30.00,1.94
4,41.76,23.66,42.18,23.64,41249,37.63,30.00,37.84,29.98,2.47
5,42.18,23.64,42.53,23.63,41249,37.84,29.98,38.11,29.97,2.30
6,42.53,23.63,42.88,23.62,41249,38.11,29.97,38.41,29.94,2.90
7,42.88,23.62,43.19,23.60,41249,38.41,29.94,38.80,29.86,2.96
8,43.19,23.60,43.46,23.58,41249,38.80,29.86,39.19,29.79,2.66
9,43.46,23.58,43.68,23.57,41249,39.19,29.79,39.62,29.73,2.14


----

# 4. Test .py functions

In [38]:
import nfl_frame_builder as nf
week_df = acquire.week(week_num)
scout_pass_block = acquire.scout_pass_block()

In [39]:
qb_hold_time, point_of_scrimmage, analysis_frames = nf.build_play_frames(game, play, week_df, nflId, scout_pass_block, player_type, v_type = 'PvB')


In [40]:
qb_hold_time

3.1

In [41]:
point_of_scrimmage

(38.31, 23.87)

In [42]:
analysis_frames

,ball_x,ball_y,ball_next_x,ball_next_y,pass_rusher,pass_rusher_x,pass_rusher_y,pass_rusher_next_x,pass_rusher_next_y,pass_rusher_a
0,39.66,23.73,40.21,23.73,41249,37.22,30.03,37.27,30.03,0.24
1,40.21,23.73,40.76,23.71,41249,37.27,30.03,37.34,30.01,0.80
2,40.76,23.71,41.29,23.69,41249,37.34,30.01,37.44,30.01,1.41
3,41.29,23.69,41.76,23.66,41249,37.44,30.01,37.63,30.00,1.94
4,41.76,23.66,42.18,23.64,41249,37.63,30.00,37.84,29.98,2.47
5,42.18,23.64,42.53,23.63,41249,37.84,29.98,38.11,29.97,2.30
6,42.53,23.63,42.88,23.62,41249,38.11,29.97,38.41,29.94,2.90
7,42.88,23.62,43.19,23.60,41249,38.41,29.94,38.80,29.86,2.96
8,43.19,23.60,43.46,23.58,41249,38.80,29.86,39.19,29.79,2.66
9,43.46,23.58,43.68,23.57,41249,39.19,29.79,39.62,29.73,2.14
